# Taking out cells

In [1]:
import sys
import os as os
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle as pkl

import scipy as scipy
import scipy.io as spio
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import signal
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy.io.matlab import mat_struct
import pickle
import random
random.seed(666)

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=True, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], scipy.io.matlab.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, scipy.io.matlab.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

## Load session data

In [2]:
meta = {}
meta[45] = {'mouse':'3C280', 'ks':'Kilosort_2024-04-12_162032', 'del_units':[594], 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[46] = {'mouse':'3C280', 'ks':'Kilosort_2024-04-12_180855', 'del_units':None, 'vis_mice':np.array(['nan', 'alpha', 'beta'])}
meta[11] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-06_154258', 'del_units':[847, 835], 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[13] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-14_120055', 'del_units':None, 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[14] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-14_122629', 'del_units':None, 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[19] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-14_140410', 'del_units':None, 'vis_mice':np.array(['nan', 'beta', 'alpha'])}
meta[20] = {'mouse':'3C290', 'ks':'Kilosort_2024-05-15_110539', 'del_units':[33], 'vis_mice':np.array(['nan', 'beta', 'alpha'])}

In [3]:
Session = 45

In [4]:
mouse = meta[Session]['mouse']
ks = meta[Session]['ks']
del_units=meta[Session]['del_units']
vis_mice = meta[Session]['vis_mice']
active_blocks = [1,2,3]


In [5]:
a_idx = np.where(vis_mice=='alpha')[0][0]
b_idx = np.where(vis_mice=='beta')[0][0]
c_idx = 3 

In [6]:
# create folder to save results
try:
    os.makedirs(f'C:\\Users\\ebukina\\Desktop\\eva\\results\\taking_out_analysis')
except:
    pass

In [7]:
# create folder to save results
try:
    os.makedirs(f'C:\\Users\\ebukina\\Desktop\\eva\\results\\taking_out_analysis\\S{Session}')
except:
    pass

### Ephys data

In [8]:
KSdir = f'L:\\everyone\\sharedDATA\\ProcessedDATA\\{mouse}\\{mouse}_S{Session}\\{ks}\\'

spiketimesfile = KSdir+"spike_times.npy"  
spiketimes = np.load(spiketimesfile)  #### all spiketimes as indexes regardless of cluster

clusterfile = KSdir+"spike_clusters.npy"
spikeclusters = np.load(clusterfile) #### cluster id for each detected spike

Clusterinfofile = KSdir+"cluster_info.tsv"
Clusterinfo = pd.read_csv(Clusterinfofile,sep='\t') #### cluster meta-data matrix

In [9]:
goodclusts = Clusterinfo['cluster_id'][np.where(Clusterinfo['group']=='good')[0]]
goods = []
for clust in goodclusts :
    goods.append(clust)
print(f'Session {Session}: {len(goods)} good clusters')

Session 45: 24 good clusters


In [10]:
goodspiketimes = {}

spikethresh = 1000 # min nb of spikes
for goodunit in goods :
    goodinds = np.where(spikeclusters==goodunit)[0]
    if goodinds.shape[0] > spikethresh : 
        goodspiketimes[goodunit] = spiketimes[goodinds]

In [11]:
len(goodspiketimes.keys())

24

### Behavior data

In [12]:
matfile =  f'L:\\everyone\\sharedDATA\\ProcessedDATA\\{mouse}\\{mouse}_S{Session}\\Behaviour.mat'
EvaBehavior = loadmat(matfile)

In [13]:
def load_behavioral_event(event_idx, time_to_idx = True):
    '''
    returns dictionary; keys - blocks; inside - array with event :
    time stamps in [sec] if time_to_idx = False
    indexes if time_to_idx = True
    '''
    event_dic = {}

    for block in np.arange(EvaBehavior['Behaviour'].shape[0]) :
        if time_to_idx:
            event_dic[block] = EvaBehavior['Behaviour'][block][event_idx]*20000
        else:
            event_dic[block] = EvaBehavior['Behaviour'][block][event_idx]
        event_dic[block] = event_dic[block].astype(int)
    
    return event_dic

In [14]:
TestSocialSampleWindowPerTrial = load_behavioral_event(12) 

### Firing rates

In [15]:
import copy

In [16]:
def fr_matrix_prep(beforesamples, aftersamples, blocks, bins, del_units=None):
    # discard wierd cells based on rasters
    goodspiketimes_copy = copy.deepcopy(goodspiketimes)

    try:
        for unit in del_units:
            del goodspiketimes_copy[unit]
    except:
        pass

    units = goodspiketimes_copy.keys()
    fr_dic = {}
   
    for block in blocks:
        n_trials = TestSocialSampleWindowPerTrial[block].shape[0]
        n_units = len(goodspiketimes_copy.keys())

        fr_matrix = np.zeros((n_trials, n_units))

        i = 0
        for unit in units: # list of good cluster_id
            
            
            for event in np.arange(TestSocialSampleWindowPerTrial[block].shape[0]) : #iterate by npoke events

                onset = TestSocialSampleWindowPerTrial[block][event,0]
                offset = TestSocialSampleWindowPerTrial[block][event,1]
                
                cond1 = np.where(goodspiketimes_copy[unit].astype(int)>=onset-beforesamples)[0]
                cond2 = np.where(goodspiketimes_copy[unit].astype(int)<=offset+aftersamples)[0]
                unitspikes = goodspiketimes_copy[unit][np.intersect1d(cond1,cond2)].astype(int)-onset

                n = np.histogram(unitspikes, bins=bins)[0][0]
                fr = n/0.25
                fr_matrix[event, i] = fr
            i+=1
                
        fr_dic[block] = fr_matrix
    
    return fr_dic

In [17]:
## find spikes in interesting behavioral window
samplplimg_rate = 20*1000 #20 kHz
beforesamples = 0*samplplimg_rate # 1 sec (expressed in samples for a 20khz sampling rate)
aftersamples = 0*samplplimg_rate # 1 sec
npoke_window = 0.25
eventlength = npoke_window*samplplimg_rate # 0.25 s

binsize = 0.25*samplplimg_rate 
bins = np.linspace(-beforesamples,aftersamples+eventlength,int((beforesamples+aftersamples+eventlength)/binsize)+1)

In [18]:
# 250 ms during the nosepoke
firing_rates_dic = fr_matrix_prep(beforesamples, aftersamples, blocks=[1,2,3], bins=bins, del_units=del_units)

In [19]:
firing_rates_dic[a_idx].shape

(30, 23)

## Classifiers

In [20]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from scipy.stats import percentileofscore
from sklearn.preprocessing import StandardScaler
import random

In [21]:
def SVM(alpha_data, beta_data, kernel = 'linear', cv=5):

    # params
    num_rows = alpha_data.shape[0]
    indices = list(range(num_rows))
    train_proportion = 6 # out of 30 trials

    # collect outcomes
    accuracy_cv5 = []

    for n in range(cv):
        test_alpha_idx = random.sample(indices, train_proportion)
        train_alpha_idx = [number for number in indices if number not in test_alpha_idx]
        test_alpha = alpha_data[test_alpha_idx]
        train_alpha = alpha_data[train_alpha_idx]

        test_beta_idx = random.sample(indices, train_proportion)
        train_beta_idx = [number for number in indices if number not in test_beta_idx]
        test_beta = beta_data[test_beta_idx]
        train_beta = beta_data[train_beta_idx]

        test_ab = np.concatenate((test_alpha,test_beta), axis=0)
        train_ab = np.concatenate((train_alpha,train_beta), axis=0)

        # Standardize the data
        scaler = StandardScaler()
        test_ab = scaler.fit_transform(test_ab)
        train_ab = scaler.fit_transform(train_ab)

        y_test = np.concatenate((np.ones(test_alpha.shape[0]),np.zeros(test_beta.shape[0])))
        y_train = np.concatenate((np.ones(train_alpha.shape[0]),np.zeros(train_beta.shape[0])))

        # randomize train
        num_rows = train_ab.shape[0]
        shuffled_indices = list(range(num_rows))
        random.shuffle(shuffled_indices)
        train_ab = train_ab[shuffled_indices]
        y_train = y_train[shuffled_indices]

        svm_classifier_cv = SVC(kernel=kernel, random_state=666)
        svm_classifier_cv.fit(train_ab, y_train)
        y_pred = svm_classifier_cv.predict(test_ab)
        accuracy_cv5.append(accuracy_score(y_test, y_pred)) 

    return np.mean(np.array(accuracy_cv5)) 

In [22]:
# data prep
alpha_data = firing_rates_dic[a_idx]
beta_data = firing_rates_dic[b_idx]
blank_data = firing_rates_dic[c_idx]

num_rows = alpha_data.shape[0]
shuffled_indices = list(range(num_rows))

random.shuffle(shuffled_indices)
alpha_data = alpha_data[shuffled_indices]

random.shuffle(shuffled_indices)
beta_data = beta_data[shuffled_indices]

num_rows = alpha_data.shape[0]
social_data = np.concatenate((alpha_data[:int(num_rows/2),:], beta_data[int(num_rows/2):,:]), axis=0)

random.shuffle(shuffled_indices)
blank_data = blank_data[shuffled_indices]

alpha_data.shape, beta_data.shape, blank_data.shape, social_data.shape

((30, 23), (30, 23), (30, 23), (30, 23))

### Taking out neurons

In [23]:
n_neurons = firing_rates_dic[a_idx].shape[1]
neurons_idx = np.arange(0,n_neurons,1)
n_neurons, neurons_idx

(23,
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22]))

In [24]:
SVM_s_vs_bl = []
SVM_a_vs_b = []

for n_take in range(n_neurons): # take out from 0 to n-1 neurons

    SVM_sbl = []
    SVM_ab = []

    mew=True
    
    for ch in range(100):
        
        # remove columns (neurons)
        take_idx = random.sample(list(neurons_idx), k=n_take)
        a_matrix = np.delete(alpha_data, take_idx, axis=1)
        b_matrix = np.delete(beta_data, take_idx, axis=1)
        bl_matrix = np.delete(blank_data, take_idx, axis=1)
        s_matrix = np.delete(social_data, take_idx, axis=1)

        if ch<=3:
            print(a_matrix.shape, b_matrix.shape, bl_matrix.shape, s_matrix.shape)

        svm_ab = SVM(a_matrix, b_matrix)
        svm_sbl = SVM(bl_matrix, s_matrix)

        SVM_ab.append(svm_ab)
        SVM_sbl.append(svm_sbl)
        

    SVM_a_vs_b.append(np.mean(np.array(SVM_ab)))
    SVM_s_vs_bl.append(np.mean(np.array(SVM_sbl)))

(30, 23) (30, 23) (30, 23) (30, 23)
(30, 23) (30, 23) (30, 23) (30, 23)
(30, 23) (30, 23) (30, 23) (30, 23)
(30, 23) (30, 23) (30, 23) (30, 23)
(30, 22) (30, 22) (30, 22) (30, 22)
(30, 22) (30, 22) (30, 22) (30, 22)
(30, 22) (30, 22) (30, 22) (30, 22)
(30, 22) (30, 22) (30, 22) (30, 22)
(30, 21) (30, 21) (30, 21) (30, 21)
(30, 21) (30, 21) (30, 21) (30, 21)
(30, 21) (30, 21) (30, 21) (30, 21)
(30, 21) (30, 21) (30, 21) (30, 21)
(30, 20) (30, 20) (30, 20) (30, 20)
(30, 20) (30, 20) (30, 20) (30, 20)
(30, 20) (30, 20) (30, 20) (30, 20)
(30, 20) (30, 20) (30, 20) (30, 20)
(30, 19) (30, 19) (30, 19) (30, 19)
(30, 19) (30, 19) (30, 19) (30, 19)
(30, 19) (30, 19) (30, 19) (30, 19)
(30, 19) (30, 19) (30, 19) (30, 19)
(30, 18) (30, 18) (30, 18) (30, 18)
(30, 18) (30, 18) (30, 18) (30, 18)
(30, 18) (30, 18) (30, 18) (30, 18)
(30, 18) (30, 18) (30, 18) (30, 18)
(30, 17) (30, 17) (30, 17) (30, 17)
(30, 17) (30, 17) (30, 17) (30, 17)
(30, 17) (30, 17) (30, 17) (30, 17)
(30, 17) (30, 17) (30, 17) (

In [25]:
n_trials = firing_rates_dic[a_idx].shape[0]
trials_idx = np.arange(0,n_trials,1)
trials_idx

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [26]:
tSVM_s_vs_bl = []
tSVM_a_vs_b = []

for n_take in range(n_trials-6): # we need at least 7 trials since test=6

    SVM_sbl = []
    SVM_ab = []
    mew=True

    for ch in range(100):
        # remove rows (trials)

        take_idx = random.sample(list(trials_idx), k=n_take)
        a_matrix = np.delete(alpha_data, take_idx, axis=0)
        b_matrix = np.delete(beta_data, take_idx, axis=0)
        bl_matrix = np.delete(blank_data, take_idx, axis=0)
        s_matrix = np.delete(social_data, take_idx, axis=0)
    
        if ch<3:
            print(a_matrix.shape, b_matrix.shape, bl_matrix.shape, s_matrix.shape)

        svm_ab = SVM(a_matrix, b_matrix)
        svm_sbl = SVM(bl_matrix, s_matrix)

        SVM_ab.append(svm_ab)
        SVM_sbl.append(svm_sbl)

    tSVM_a_vs_b.append(np.mean(np.array(SVM_ab)))
    tSVM_s_vs_bl.append(np.mean(np.array(SVM_sbl)))
  

(30, 23) (30, 23) (30, 23) (30, 23)
(30, 23) (30, 23) (30, 23) (30, 23)
(30, 23) (30, 23) (30, 23) (30, 23)
(29, 23) (29, 23) (29, 23) (29, 23)
(29, 23) (29, 23) (29, 23) (29, 23)
(29, 23) (29, 23) (29, 23) (29, 23)
(28, 23) (28, 23) (28, 23) (28, 23)
(28, 23) (28, 23) (28, 23) (28, 23)
(28, 23) (28, 23) (28, 23) (28, 23)
(27, 23) (27, 23) (27, 23) (27, 23)
(27, 23) (27, 23) (27, 23) (27, 23)
(27, 23) (27, 23) (27, 23) (27, 23)
(26, 23) (26, 23) (26, 23) (26, 23)
(26, 23) (26, 23) (26, 23) (26, 23)
(26, 23) (26, 23) (26, 23) (26, 23)
(25, 23) (25, 23) (25, 23) (25, 23)
(25, 23) (25, 23) (25, 23) (25, 23)
(25, 23) (25, 23) (25, 23) (25, 23)
(24, 23) (24, 23) (24, 23) (24, 23)
(24, 23) (24, 23) (24, 23) (24, 23)
(24, 23) (24, 23) (24, 23) (24, 23)
(23, 23) (23, 23) (23, 23) (23, 23)
(23, 23) (23, 23) (23, 23) (23, 23)
(23, 23) (23, 23) (23, 23) (23, 23)
(22, 23) (22, 23) (22, 23) (22, 23)
(22, 23) (22, 23) (22, 23) (22, 23)
(22, 23) (22, 23) (22, 23) (22, 23)
(21, 23) (21, 23) (21, 23) (

In [27]:
SVM_take_out_dic = {}
SVM_take_out_dic['SVM_s_vs_bl'] = SVM_s_vs_bl
SVM_take_out_dic['SVM_a_vs_b'] = SVM_a_vs_b
SVM_take_out_dic['tSVM_s_vs_bl'] = tSVM_s_vs_bl
SVM_take_out_dic['tSVM_a_vs_b'] = tSVM_a_vs_b

In [28]:
save_path = f'C:\\Users\\ebukina\\Desktop\\eva\\results\\taking_out_analysis\\S{Session}\\{mouse}_S{Session}_SVM_take_out_dic.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(SVM_take_out_dic, f)

## Shuffle control

In [29]:
concat_data = np.concatenate((alpha_data,beta_data,blank_data), axis=0)
concat_data.shape

(90, 23)

In [30]:
n_rows = concat_data.shape[0]
rows_idx = np.arange(0,n_rows,1)
n_rows, rows_idx

(90,
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89]))

In [31]:
shuffled_indices = list(range(n_rows))
random.shuffle(shuffled_indices)
concat_data_sh = concat_data[shuffled_indices]

In [32]:
sh_neurons_sbl_list = []
sh_neurons_ab_list = []

for sh in range(100):
    # making shuffle data
    take_idx = random.sample(list(rows_idx), k=30)
    alpha_matrix = concat_data_sh[take_idx]
    take_idx = random.sample(list(rows_idx), k=30)
    beta_matrix = concat_data_sh[take_idx]
    take_idx = random.sample(list(rows_idx), k=30)
    social_matrix = concat_data_sh[take_idx]
    take_idx = random.sample(list(rows_idx), k=30)
    blank_matrix = concat_data_sh[take_idx]

    SVM_s_vs_bl = []
    SVM_a_vs_b = []


    for n_take in range(n_neurons): # take out from 0 to n-1 neurons

        SVM_sbl = []
        SVM_ab = []
        
        for ch in range(100):
            
            # remove columns (neurons)
            take_idx = random.sample(list(neurons_idx), k=n_take)
            a_matrix = np.delete(alpha_matrix, take_idx, axis=1)
            b_matrix = np.delete(beta_matrix, take_idx, axis=1)
            bl_matrix = np.delete(blank_matrix, take_idx, axis=1)
            s_matrix = np.delete(social_matrix, take_idx, axis=1)

            svm_ab = SVM(a_matrix, b_matrix)
            svm_sbl = SVM(bl_matrix, s_matrix)

            SVM_ab.append(svm_ab)
            SVM_sbl.append(svm_sbl)
            
        SVM_s_vs_bl.append(np.mean(np.array(SVM_sbl)))
        SVM_a_vs_b.append(np.mean(np.array(SVM_ab)))

    sh_neurons_sbl_list.append(SVM_s_vs_bl)
    sh_neurons_ab_list.append(SVM_a_vs_b)

KeyboardInterrupt: 

In [ ]:
# delete trials
sh_trials_sbl_list = []
sh_trils_ab_list = []

for sh in range(100):
    # making shuffle data
    take_idx = random.sample(list(rows_idx), k=30)
    alpha_matrix = concat_data_sh[take_idx]
    take_idx = random.sample(list(rows_idx), k=30)
    beta_matrix = concat_data_sh[take_idx]
    take_idx = random.sample(list(rows_idx), k=30)
    social_matrix = concat_data_sh[take_idx]
    take_idx = random.sample(list(rows_idx), k=30)
    blank_matrix = concat_data_sh[take_idx]

    SVM_s_vs_bl = []
    SVM_a_vs_b = []


    for n_take in range(n_trials-6): # take out from 0 to n-1 neurons

        SVM_sbl = []
        SVM_ab = []
        
        for ch in range(100):
            
            # remove columns (neurons)
            take_idx = random.sample(list(trials_idx), k=n_take)
            a_matrix = np.delete(alpha_matrix, take_idx, axis=0)
            b_matrix = np.delete(beta_matrix, take_idx, axis=0)
            bl_matrix = np.delete(blank_matrix, take_idx, axis=0)
            s_matrix = np.delete(social_matrix, take_idx, axis=0)

            svm_ab = SVM(a_matrix, b_matrix)
            svm_sbl = SVM(bl_matrix, s_matrix)

            SVM_ab.append(svm_ab)
            SVM_sbl.append(svm_sbl)
            
        SVM_s_vs_bl.append(np.mean(np.array(SVM_sbl)))
        SVM_a_vs_b.append(np.mean(np.array(SVM_ab)))

    sh_trials_sbl_list.append(SVM_s_vs_bl)
    sh_trils_ab_list.append(SVM_a_vs_b)

In [ ]:
sh_dic = {}
sh_dic['sh_neurons_sbl_list'] = sh_neurons_sbl_list
sh_dic['sh_neurons_ab_list'] = sh_neurons_ab_list
sh_dic['sh_trials_sbl_list'] = sh_trials_sbl_list
sh_dic['sh_trils_ab_list'] = sh_trils_ab_list

In [ ]:
save_path = f'C:\\Users\\ebukina\\Desktop\\eva\\results\\taking_out_analysis\\S{Session}\\{mouse}_S{Session}_sh_dic.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(sh_dic, f)